In [ ]:
using Revise
using Pkg; Pkg.activate(".")

In [ ]:
using Unitful
using PotentialLearning
using Random: randperm
using JLD2
using InteratomicPotentials
using AtomsBase, AtomsCalculators
using Statistics
using CairoMakie, ColorSchemes

In [ ]:
# from subsampling_dpp.jl in PL.jl examples
function concat_dataset(confs::Vector{DataSet})
    N = length(confs)
    confs_vec = [[confs[i][j] for j = 1:length(confs[i])] for i = 1:N]
    confs_all = reduce(vcat, confs_vec)
    return DataSet(confs_all)
end

In [ ]:
ace = ACE(species            = [:Hf],
          body_order         = 4,
          polynomial_degree  = 10,
          wL                 = 1.5,
          csp                = 1.0,
          r0                 = 2.15,
          rcutoff            = 5.0)

In [ ]:
pristine_8x_config_sets = Vector{DataSet}()
frenkel_8x_config_sets = Vector{DataSet}()
dilute_8x_config_sets = Vector{DataSet}()

# large_8x frenkel
for i in 1:5
    println(i)
    configs = load_data("./data/pod_Hf_frenkel_large/large_8x_frenkel_$(i).xyz", ExtXYZ(u"eV", u"Å"))
    push!(frenkel_8x_config_sets, configs)
end

# large_8x dilute frenkel
for i in 1:5
    println(i)
    configs = load_data("./data/pod_Hf_frenkel_large/large_dilute_frenkel_$(i).xyz", ExtXYZ(u"eV", u"Å"))
    push!(dilute_8x_config_sets, configs)
end

# large_8x pristine
for i in 1:5
    println(i)
    configs = load_data("./data/pod_Hf_frenkel_large/large_pristine_$(i).xyz", ExtXYZ(u"eV", u"Å"))
    push!(pristine_8x_config_sets, configs)
end

In [ ]:
pristine_8x_configs = concat_dataset(pristine_8x_config_sets)
frenkel_8x_configs = concat_dataset(frenkel_8x_config_sets)
dilute_8x_configs = concat_dataset(dilute_8x_config_sets)

In [ ]:
edescr_pristine_8x = compute_local_descriptors(pristine_8x_configs,ace)
fdescr_pristine_8x = compute_local_descriptors(pristine_8x_configs,ace)

In [ ]:
edescr_frenkel_8x = compute_local_descriptors(frenkel_8x_configs,ace)
fdescr_frenkel_8x = compute_local_descriptors(frenkel_8x_configs,ace)

In [ ]:
edescr_dilute_frenkel_8x = compute_local_descriptors(dilute_8x_configs,ace)
fdescr_dilute_frenkel_8x = compute_local_descriptors(dilute_8x_configs,ace)

In [ ]:
pristine_8x_ds = DataSet(pristine_8x_configs .+ edescr_pristine_8x .+ fdescr_pristine_8x)
frenkel_8x_ds = DataSet(frenkel_8x_configs .+ edescr_frenkel_8x .+ fdescr_frenkel_8x)
dilute_8x_ds = DataSet(dilute_8x_configs .+ edescr_dilute_frenkel_8x .+ fdescr_dilute_frenkel_8x)

In [ ]:
save("large_8x_data.jld2", Dict("pristine_8x_ds" => pristine_8x_ds,
                                "frenkel_8x_ds" => frenkel_8x_ds,
                                "dilute_8x_ds" => dilute_8x_ds))